In [3]:
from dotenv import load_dotenv
import os
from pathlib import Path

env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

# reference: https://stackoverflow.com/questions/16924471/difference-between-os-getenv-and-os-environ-get
try:
    api_key = os.environ["GOOGLE_API_KEY"]
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' in .env file. Details: {e}")

✅ Gemini API key setup complete.


In [4]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


- 사용자에게 지난 7주일 동안 며칠에 운동을 나갔는지 묻는다 
- 스크래핑 해서 와드 정보 가져옫다.

In [5]:
# customer_facing_agent
get_wod_agent = Agent(
    name="GetWodAgent",
    model="gemini-2.5-flash-lite",
    instruction="""
        To customer user, politely inquire which date the user has attended crossfit WOD out of the prior 7 days. 
        Based on the response of the user, retrieve daily wod content only from official crossfit website, https://www.crossfit.com/ 
        The url is very straight-forward for each date.
        For example, if user attended wod on 2025 Nov 24, you should look up "crossift.com/251124
        Unless specified, the default year is this year.
        "
 """,
 tools = [google_search],
 output_key="get_wod_agent_output",
)
print(">> get_wod_agent created.")

>> get_wod_agent created.


In [6]:
# per_wod_analysis_agent
analyze_wod_agent = Agent(
    name="AnalyzeWodAgent",
    model="gemini-2.5-flash-lite",
    instruction="""
        You are an experienced researcher in functional anatomy, and exercise physiology specialized in CrossFit movements.
        Based on {get_wod_agent_output}, analyze the cumulative physiological adaptations 
        assuming these WODs were performed sequentially in different days. 
        Break down each wod's effect by MGW modalities and specify the targeted muscle groups.
        Present the summary as a bulleted list with 3-5 key pointsn for each different wod.
 """,
 output_key="analyze_wod_output"
)
print(">> analyze_wod_agent created")

>> analyze_wod_agent created


In [7]:
recommend_agent = Agent(
    name="RecommendAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Review {analyze_wod_output}, and suggest 3-5 supplementary exercises or stretches 
    to help the member maintain a balanced physique and recover overused muscle areas. 
    Present the result in 3 to 5 bullet points.
    """,
    tools=[google_search],
    output_key="recommend_output"
)
print(">> recommend_agent creeated")

>> recommend_agent creeated


In [9]:

root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[get_wod_agent, analyze_wod_agent, recommend_agent],
)

print(">> parallel and sequential agents created")

>> parallel and sequential agents created


In [59]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("I went to crossfit on Nov 24 and Nov 25.")


 ### Created new session: debug_session_id

User > I went to crossfit on Nov 24 and Nov 25.
GetWodAgent > For November 24, 2025, the WOD was: 7 sets, each for load: 1 push jerk + 1 split jerk. Post loads to the comments.

For November 25, 2025, the WOD was "Cindy": Complete as many rounds and reps as possible in 20 minutes of: 5 pull-ups, 10 push-ups, 15 air squats.
AnalyzeWodAgent > Here's an analysis of the physiological adaptations from the sequential CrossFit WODs on November 24th and 25th, 2025:

**November 24, 2025: 7 Sets of 1 Push Jerk + 1 Split Jerk (for Load)**

*   **MGW Modalities:** Primarily focuses on **Strength and Power**. The heavy, low-rep nature targets the neuromuscular system for maximal force production and rapid contraction. It also involves significant **Skill and Technique** due to the complex coordination required for the jerks.
*   **Targeted Muscle Groups:**
    *   **Legs:** Quadriceps, hamstrings, glutes (for the dip and drive phase).
    *   **Shoulders

In [2]:
%pip install markdown xhtml2pdf

  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 665.9/665.9 kB 8.9 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.0 MB/s  0:00:00
Using cached webencodings-0.5.1-py2.py3-none-any.whl (11 kB)
  Created wheel for pycairo: filename=pycairo-1.29.0-cp311-cp311-macosx_15_0_arm64.whl size=112529 sha256=46c6f74c7b1e85fa5569cb2aef203acb9ab45933098d3c8ed05e25b337b24cc3
  Stored in directory: /Users/yujinchung/Library/Caches/pip/wheels/60/8f/09/dd419a45124a1c8a273853c3f7501c696e0c4951875f6e8203
Successfully built pycairo
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18/18 [xhtml2pdf]18 [pyHanko]]]
Note: you may need to restart the kernel to use updated pack

In [10]:
import sys
import io
import markdown
from xhtml2pdf import pisa

def save_markdown_to_pdf(markdown_text, output_filename):
    # Convert Markdown to HTML
    html_text = markdown.markdown(markdown_text)
    
    # Add some basic styling
    full_html = f"""
    <html>
    <head>
        <style>
            body {{ font-family: sans-serif; font-size: 12px; }}
            h1, h2, h3 {{ color: #333; }}
            p {{ line-height: 1.5; }}
            ul {{ margin-bottom: 10px; }}
        </style>
    </head>
    <body>
        {html_text}
    </body>
    </html>
    """
    
    # Generate PDF
    with open(output_filename, "wb") as result_file:
        pisa_status = pisa.CreatePDF(
            io.BytesIO(full_html.encode("utf-8")),
            dest=result_file
        )
    
    if pisa_status.err:
        print(f"Error generating PDF: {pisa_status.err}")
    else:
        print(f"PDF saved successfully to {output_filename}")

# Capture stdout
old_stdout = sys.stdout
sys.stdout = mystdout = io.StringIO()

try:
    # Re-run the agent to capture the output
    runner = InMemoryRunner(agent=root_agent)
    response = await runner.run_debug("I went to crossfit on Nov 24 and Nov 25.")
finally:
    sys.stdout = old_stdout

# Get the captured text
captured_output = mystdout.getvalue()

# Print it back to console so the user still sees it
print(captured_output)

# Save to PDF
save_markdown_to_pdf(captured_output, "crossfit_report.pdf")


 ### Created new session: debug_session_id

User > I went to crossfit on Nov 24 and Nov 25.
GetWodAgent > For November 24th, the WOD was "7 sets, each for load: 1 push jerk + 1 split jerk. Post loads to the comments."

For November 25th, the WOD was "Cindy. Complete as many rounds and reps as possible in 20 minutes of: 5 pull-ups 10 push-ups 15 air squats. Post round and reps to the comments."
AnalyzeWodAgent > Here's an analysis of the physiological adaptations from the consecutive CrossFit WODs on November 24th and 25th:

**November 24th: 7 Sets, Each for Load: 1 Push Jerk + 1 Split Jerk**

*   **MGW Modality:** Primarily **Strength & Power**. This WOD focuses on maximal or near-maximal attempts to move a heavy load.
*   **Targeted Muscle Groups:**
    *   **Legs (Quadriceps, Hamstrings, Glutes):** Crucial for the dip and drive in the push jerk and the initial drive in the split jerk.
    *   **Shoulders (Deltoids):** Primary movers for pressing the weight overhead.
    *   **Tricep